# Twitter Streaming APP
In order to use all of this though, we need to setup a Developer API acocunt with Twitter and create an application to get credentials. Review the video for instructions on how to do this or if you are already familiar with it, just get the credentials from: 

    https://apps.twitter.com/
    
Once you have that you also need to install python-twitter, a python library to connect your Python to the twitter dev account.

You probably won't be able to run this example and then previous in the same notebook, you need to restart you kernel.

Let's get started!

Begin by running the TweetRead.py file. Make sure to add your own IP Adress and your credential keys.

In [1]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

23/08/23 21:03:06 WARN Utils: Your hostname, Blade-15-Base-Model resolves to a loopback address: 127.0.1.1; using 192.168.0.23 instead (on interface wlo1)
23/08/23 21:03:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 21:03:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

/home/ilyes/.local/lib/python3.10/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
/home/ilyes/.local/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
socket_stream = ssc.socketTextStream("127.0.0.1", 9998)

In [5]:
lines = socket_stream.window( 20 )

In [6]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [7]:
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ) ) # Registers to a table.

__________
### Now run TweetRead.py
__________

In [8]:
ssc.start()    

23/08/23 21:03:43 ERROR JobScheduler: Error running job streaming job 1692817420000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/streaming/dstream.py", line 236, in func
    return old_func(rdd)  # type: ignore[call-arg, arg-type]
  File "/tmp/ipykernel_4072/1757346720.py", line 7, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 115, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 1276, in createDataFrame
    return self._create_dataframe(
  File "/home/ilyes/.local/lib/python3.10/site-package

In [9]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

23/08/23 20:58:31 ERROR JobScheduler: Error running job streaming job 1692817110000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/streaming/dstream.py", line 236, in func
    return old_func(rdd)  # type: ignore[call-arg, arg-type]
  File "/tmp/ipykernel_7185/1757346720.py", line 7, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 115, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 1276, in createDataFrame
    return self._create_dataframe(
  File "/home/ilyes/.local/lib/python3.10/site-package

In [10]:
count = 0
while count < 10:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    sns.plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=top_10_df)
    sns.plt.show()
    count = count + 1

23/08/23 20:58:51 ERROR JobScheduler: Error running job streaming job 1692817130000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/streaming/dstream.py", line 236, in func
    return old_func(rdd)  # type: ignore[call-arg, arg-type]
  File "/tmp/ipykernel_7185/1757346720.py", line 7, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 115, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 1276, in createDataFrame
    return self._create_dataframe(
  File "/home/ilyes/.local/lib/python3.10/site-package

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `tweets` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 23;
'Project ['tag, 'count]
+- 'UnresolvedRelation [tweets], [], false


23/08/23 20:59:01 ERROR JobScheduler: Error running job streaming job 1692817140000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/streaming/dstream.py", line 236, in func
    return old_func(rdd)  # type: ignore[call-arg, arg-type]
  File "/tmp/ipykernel_7185/1757346720.py", line 7, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 115, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/ilyes/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 1276, in createDataFrame
    return self._create_dataframe(
  File "/home/ilyes/.local/lib/python3.10/site-package

In [9]:
ssc.stop()

23/08/23 21:04:50 WARN ReceiverSupervisorImpl: Receiver has been stopped
Exception in thread "receiver-supervisor-future-2" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(